In [1]:
#initial imports
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np 
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import numpy as np

In [2]:
#read in incidence csv from github

mortality_df = pd.read_csv("https://raw.githubusercontent.com/robyndook/Cancer_Treatment_Centers_California/main/Resources/Race_Mortality_Cancer_DB.csv")                         
#get rid of commas in columns
mortality_df = mortality_df.replace(',','', regex=True)
mortality_df.head()     

,average_annual_count,recent_trend,primary_state_name,cancer,race_ethnicity,sex,age,age_adjusted_rate,five_year_trend_rates,county_state,index_of_medical_underservice_score,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019
0,181,stable,Alabama,All Cancer Sites,White (includes Hispanic),All Sexes,All Ages,213.2,213.2,Walker County AL,61.0,64931,64533,63895,63669,63521
1,180,stable,Alabama,All Cancer Sites,White Non-Hispanic,All Sexes,All Ages,213.9,213.9,Walker County AL,61.0,64931,64533,63895,63669,63521
2,10,stable,Alabama,All Cancer Sites,Black (includes Hispanic),All Sexes,All Ages,281.1,281.1,Walker County AL,61.0,64931,64533,63895,63669,63521
3,32,stable,Alabama,All Cancer Sites,White (includes Hispanic),All Sexes,All Ages,211.2,211.2,Crenshaw County AL,58.0,13850,13916,13858,13825,13772
4,32,stable,Alabama,All Cancer Sites,White Non-Hispanic,All Sexes,All Ages,214.0,214.0,Crenshaw County AL,58.0,13850,13916,13858,13825,13772


In [3]:
#check dtypes
mortality_df.dtypes

average_annual_count                     int64
recent_trend                            object
primary_state_name                      object
cancer                                  object
race_ethnicity                          object
sex                                     object
age                                     object
age_adjusted_rate                       object
five_year_trend_rates                   object
county_state                            object
index_of_medical_underservice_score    float64
popestimate2015                          int64
popestimate2016                          int64
popestimate2017                          int64
popestimate2018                          int64
popestimate2019                          int64
dtype: object

In [4]:
#get count of rows
mortality_df.count()

average_annual_count                   7341
recent_trend                           7341
primary_state_name                     7341
cancer                                 7341
race_ethnicity                         7341
sex                                    7341
age                                    7341
age_adjusted_rate                      7341
five_year_trend_rates                  7341
county_state                           7341
index_of_medical_underservice_score    6439
popestimate2015                        7341
popestimate2016                        7341
popestimate2017                        7341
popestimate2018                        7341
popestimate2019                        7341
dtype: int64

In [5]:
#get count of values for race
mortality_df["race_ethnicity"].value_counts()

White (includes Hispanic)                       2414
  White Non-Hispanic                            2311
Black (includes Hispanic)                       1012
Hispanic (any race)                              572
  White Hispanic                                 560
Asian / Pacific Islander (includes Hispanic)     286
Amer. Indian / AK Native (includes Hispanic)     186
Name: race_ethnicity, dtype: int64

In [6]:
#get count of NA values for MUA
mortality_df["index_of_medical_underservice_score"].isna().sum()

902

In [7]:
#drop rows where MUA index is null
mortality_df = mortality_df[mortality_df["index_of_medical_underservice_score"].isna() == False]

In [8]:
#get count of rows
mortality_df.count()

average_annual_count                   6439
recent_trend                           6439
primary_state_name                     6439
cancer                                 6439
race_ethnicity                         6439
sex                                    6439
age                                    6439
age_adjusted_rate                      6439
five_year_trend_rates                  6439
county_state                           6439
index_of_medical_underservice_score    6439
popestimate2015                        6439
popestimate2016                        6439
popestimate2017                        6439
popestimate2018                        6439
popestimate2019                        6439
dtype: int64

In [9]:
#create target 
y = mortality_df[['age_adjusted_rate']]

#change dtype to numeric
y['age_adjusted_rate'] = y['age_adjusted_rate'].astype(float)
y.head()

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,age_adjusted_rate
0,213.2
1,213.9
2,281.1
3,211.2
4,214.0


In [10]:
#create feautures variable holding ages and mua score
X = mortality_df[["race_ethnicity", "index_of_medical_underservice_score"]]
X.head()

,race_ethnicity,index_of_medical_underservice_score
0,White (includes Hispanic),61.0
1,White Non-Hispanic,61.0
2,Black (includes Hispanic),61.0
3,White (includes Hispanic),58.0
4,White Non-Hispanic,58.0


In [11]:
#create dummy variables for race
X = pd.get_dummies(X, columns=["race_ethnicity"])
X.head()

,index_of_medical_underservice_score,race_ethnicity_Amer. Indian / AK Native (includes Hispanic),race_ethnicity_Asian / Pacific Islander (includes Hispanic),race_ethnicity_Black (includes Hispanic),race_ethnicity_Hispanic (any race),race_ethnicity_White (includes Hispanic),race_ethnicity_ White Hispanic,race_ethnicity_ White Non-Hispanic
0,61.0,0,0,0,0,1,0,0
1,61.0,0,0,0,0,0,0,1
2,61.0,0,0,1,0,0,0,0
3,58.0,0,0,0,0,1,0,0
4,58.0,0,0,0,0,0,0,1


In [12]:
#get shape of target array
y.shape

(6439, 1)

In [13]:
#get shape of feautures array
X.shape

(6439, 8)

In [14]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
#create linear regression model using sklearn
model = LinearRegression()

In [16]:
#fit the model with training data
regression_race_MUA_mortality = model.fit(X_train, y_train)

In [17]:
#Create predictions array using linear regression
y_pred = regression_race_MUA_mortality.predict(X_test)
y_pred

array([[184.91673591],
       [169.12737059],
       [168.30007591],
       ...,
       [166.40931631],
       [166.27143386],
       [166.71265769]])

In [18]:
#get R2 value from linear regression
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.38342228808921086

In [19]:
#run regression again with statsmodels
#use statsmodels to add constant to X
#add constant to X
X1 = sm.add_constant(X)

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [20]:
#fit linear regression model
model = sm.OLS(y, X1).fit()

In [21]:
#view model summary with statsmodels
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      age_adjusted_rate   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.264
Method:                 Least Squares   F-statistic:                     330.1
Date:                Sun, 29 May 2022   Prob (F-statistic):               0.00
Time:                        21:04:27   Log-Likelihood:                -33115.
No. Observations:                6439   AIC:                         6.625e+04
Df Residuals:                    6431   BIC:                         6.630e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [22]:
#fit the model with training data
#decision tree regression
regressor_DT = DecisionTreeRegressor()
regressor_DT.fit(X_train, y_train)

DecisionTreeRegressor()

In [23]:
#predict
y_pred = regressor_DT.predict(X_test)
y_pred

array([195.4       , 207.94285714, 169.79859155, ..., 169.23617021,
       167.68671329, 163.46956522])

In [24]:
#calculate r2 score
r2_score(y_test, y_pred)

0.2876292623704735